In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl
hl.init(tmp_dir='/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/temp', spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '30G'}, default_reference='GRCh38') 

2021-12-28 14:02:50 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.2
SparkUI available at http://p0603.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.79-f141af259254
LOGGING: writing to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/analysis/burden-and-family/hail-20211228-1402-0.2.79-f141af259254.log


In [2]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()
import openpyxl


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

from scipy import stats

output_notebook() 

Loading BokehJS ...

Loading BokehJS ...

## 1. Load large matrix table, annotate with GTS phenotype, remove samples from families and run PCA

In [5]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/full.mt')

In [6]:
pheno = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv', impute = True, key='ID', delimiter = ',', quote ="\"")

2021-12-18 18:09:41 Hail: INFO: Reading table to impute column types
2021-12-18 18:09:47 Hail: INFO: Finished type imputation            (0 + 1) / 1]
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading field 'kinship' as type str (imputed)
  Loading field 'disease' as type str (imputed)
  Loading field 'phenotype' as type str (imputed)
  Loading field 'add_pheno' as type str (imputed)
  Loading field 'heavy_tics' as type str (imputed)
  Loading field 'heavy_tics_familial' as type str (imputed)
  Loading field 'GTS_ASD_group' as type str (imputed)
  Loading field 'nonCTD' as type str (imputed)


In [7]:
mt = mt.annotate_cols(phenotypes = pheno[mt.s])

In [9]:
mt = mt.filter_cols((mt.s.contains('HG')) | (mt.s.contains('B')) | (mt.s.contains('NA')) | (mt.phenotypes.family == '.'))

In [11]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [ ]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/gts-for-burden.mt') #this has been deleted

In [4]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/gts-for-burden.mt')

In [5]:
mt = mt.annotate_cols(group = hl.if_else(mt.s.contains('B'), 'sport', hl.if_else(((mt.s.contains('NA')) | (mt.s.contains('HG'))), '1kg', 'GTS')))

In [37]:
subset = mt.sample_rows(0.001)

In [38]:
subset.aggregate_cols(hl.agg.counter(subset.group)) 

frozendict({'1kg': 98, 'GTS': 40, 'sport': 100})

In [24]:
subset.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-for-pca')

2021-12-19 11:55:30 Hail: INFO: wrote matrix table with 2703 rows and 236 columns in 14999 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-for-pca
    Total size: 24.82 MiB
    * Rows/entries: 24.80 MiB
    * Columns: 20.70 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  11 rows (74.32 KiB)


In [25]:
subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-for-pca')

In [26]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(subset.GT)
subset = subset.annotate_cols(scores = pcs[subset.s].scores)

subset.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-with-pca')

2021-12-19 11:55:45 Hail: INFO: hwe_normalize: found 2608 variants after filtering out monomorphic sites.
2021-12-19 11:56:26 Hail: INFO: pca: running PCA with 10 components...) / 14999]
2021-12-19 12:22:38 Hail: INFO: wrote matrix table with 2703 rows and 236 columns in 14999 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-with-pca


In [39]:
subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-with-pca')

In [40]:
p = hl.plot.scatter(subset.scores[2],
                    subset.scores[3],
                    label=subset.group,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

## 2. Only keep intragenic variants and remove outliers from main mt

In [6]:
mt = mt.filter_rows(mt.within_gene == hl.empty_array(hl.tstr), keep = False)

In [7]:
mt = mt.filter_cols((mt.s != 'WGS_5') & (mt.s != 'WGS_8'))

## 3. Run sample and variant QC, make plots

In [8]:
mt.naive_coalesce(2500).write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-repart.mt')

2021-12-19 13:22:21 Hail: INFO: wrote matrix table with 1752554 rows and 236 columns in 2500 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-repart.mt
    Total size: 13.66 GiB
    * Rows/entries: 13.66 GiB
    * Columns: 20.70 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  7295 rows (54.73 MiB)


In [11]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-repart.mt')

In [12]:
mt = hl.sample_qc(mt)
mt = hl.variant_qc(mt)

In [13]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/for-burden.mt')

2021-12-19 13:29:05 Hail: INFO: wrote matrix table with 1752554 rows and 236 columns in 2500 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/for-burden.mt


In [15]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/for-burden.mt')

In [4]:
mt.count()

(1752554, 236)

In [5]:
mt = mt.filter_cols(mt.s != 'WGS_6827')

In [6]:
mt.aggregate_cols(hl.agg.counter(mt.group)) 

frozendict({'1kg': 98, 'GTS': 37, 'sport': 100})

In [16]:
#check gq group

mt = mt.annotate_rows(variant_qc_per_grop = hl.agg.group_by(mt.group, hl.agg.stats(mt.GQ)))

In [18]:
p = hl.plot.histogram(mt.variant_qc_per_grop['1kg'].mean, range=(10,100), legend='Mean variant GQ 1kg')
show(p)

In [21]:
p2 = hl.plot.histogram(mt.variant_qc_per_grop['sport'].mean, range=(10,100), legend='Mean variant GQ sport')
show(p2)

In [29]:
p3 = hl.plot.histogram(mt.variant_qc_per_grop.get('GTS', hl.struct(mean = 0.0, stdev = 0.0, min = 0.0, max = 0.0, n = 0, sum = 0)).mean, range=(10,100), legend='Mean variant GQ gts')
show(p3)

## 3. Explode rows, group by gene and calculate multiple burden statistics

In [8]:
mt = mt.explode_rows(mt.within_gene)
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-exploded.mt')

2021-12-19 21:49:05 Hail: INFO: wrote matrix table with 1994489 rows and 235 columns in 2500 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-exploded.mt


In [9]:
#annotate each variant with a burden score:
mt = mt.annotate_rows(burden_maf = hl.if_else(hl.is_defined(mt.gnomad_v_3_1.freq.AF[2]),
                                                  hl.if_else((mt.gnomad_v_3_1.freq.AF[2] == 0), 5, -hl.log10(mt.gnomad_v_3_1.freq.AF[2])), 5),
                      burden_cadd = (2**mt.cadd.cadd_score)) #this is to proiritise variants with high deleteriousness

In [10]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-with-burden-scores.mt')

2021-12-19 21:51:10 Hail: INFO: wrote matrix table with 1994489 rows and 235 columns in 2500 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-with-burden-scores.mt


In [4]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/burden-and-family/to-delete-with-burden-scores.mt')

In [5]:
per_gene = mt.group_rows_by(mt.within_gene).aggregate(
    burden_cadd_phred = hl.agg.sum(mt.GT.n_alt_alleles()*mt.cadd.cadd_score))

In [6]:
per_gene.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/scores.mt')

2021-12-27 13:06:10 Hail: INFO: Ordering unsorted dataset with network shuffle0]
2021-12-27 13:06:29 Hail: INFO: wrote matrix table with 36377 rows and 235 columns in 2500 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/scores.mt
    Total size: 20.77 MiB
    * Rows/entries: 20.72 MiB
    * Columns: 45.02 KiB
    * Globals: 11.00 B
    * Smallest partition: 15 rows (1.43 KiB)
    * Largest partition:  16 rows (17.59 KiB)


In [7]:
per_gene = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/scores.mt')

In [9]:
per_gene = per_gene.annotate_rows(stat = hl.agg.group_by(per_gene.group, hl.agg.stats(per_gene.burden_cadd_phred)))

## Run actual burden tests

In [3]:
gene_lists = pd.read_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/burden-and-family/gene-lists-burden-and-family.csv', delimiter = ';')

In [4]:
gene_lists = gene_lists.fillna('')

In [5]:
genes = []

for i in gene_lists.columns:
    genes.append(list(gene_lists[i]))

In [6]:
per_gene = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/scores.mt')

In [7]:
list(gene_lists.columns)

['Cerebral_Cortex',
 'Basal_Ganglia',
 'Hypothalmus',
 'Thalmus',
 'Midbrain',
 'Dopaminergiczna',
 'Serotoninergiczna',
 'Noradrenergiczna',
 'GABA',
 'SLITRK',
 'all_neurotransmitters',
 'Gutaminian',
 'Serotonina_Kannabinoidy',
 'Dopamina_Acetylocholina',
 'geny_GTS',
 'Synapsy',
 'Tryptofan',
 'Ligand_receptor',
 'calcium',
 'androgenowy',
 'addiction']

In [8]:
per_gene = per_gene.annotate_rows(cc = hl.literal(genes[0]).contains(per_gene.within_gene),
                                  bg = hl.literal(genes[1]).contains(per_gene.within_gene),
                                  hp = hl.literal(genes[2]).contains(per_gene.within_gene),
                                  t = hl.literal(genes[3]).contains(per_gene.within_gene),
                                  mb = hl.literal(genes[4]).contains(per_gene.within_gene),
                                  dop = hl.literal(genes[5]).contains(per_gene.within_gene),
                                  ser = hl.literal(genes[6]).contains(per_gene.within_gene),
                                  nor = hl.literal(genes[7]).contains(per_gene.within_gene),
                                  gaba = hl.literal(genes[8]).contains(per_gene.within_gene),
                                  slit = hl.literal(genes[9]).contains(per_gene.within_gene),
                                  nt = hl.literal(genes[10]).contains(per_gene.within_gene),
                                  glut = hl.literal(genes[11]).contains(per_gene.within_gene),
                                  sc = hl.literal(genes[12]).contains(per_gene.within_gene),
                                  da = hl.literal(genes[13]).contains(per_gene.within_gene),
                                  gts = hl.literal(genes[14]).contains(per_gene.within_gene),
                                  syn = hl.literal(genes[15]).contains(per_gene.within_gene),
                                  trp = hl.literal(genes[16]).contains(per_gene.within_gene),
                                  lr = hl.literal(genes[17]).contains(per_gene.within_gene),
                                  ca = hl.literal(genes[18]).contains(per_gene.within_gene),
                                  andr = hl.literal(genes[19]).contains(per_gene.within_gene),
                                  add = hl.literal(genes[20]).contains(per_gene.within_gene))

In [9]:
filters = [per_gene.cc, per_gene.bg, per_gene.hp, per_gene.t, per_gene.mb,
           per_gene.dop, per_gene.ser, per_gene.nor, per_gene.gaba, per_gene.slit, 
           per_gene.nt, per_gene.glut, per_gene.sc, per_gene.da, per_gene.gts, per_gene.syn, per_gene.trp,
           per_gene.lr, per_gene.ca, per_gene.andr, per_gene.add]                        

In [10]:
groups = per_gene.group.collect()

In [11]:
results_list = []
for f in filters: 
    results = per_gene.filter_rows(f)
    results = results.annotate_cols(result_score = hl.agg.sum(results['burden_cadd_phred']))
    results_list.append(results.result_score.collect())

In [13]:
from scipy import stats

t_test_gts_sport = []
t_test_gts_1kg = []
t_test_sport_1kg = []
gts_mean = []
gts_std = []
kg_mean = []
kg_std = []
sport_mean = []
sport_std = []

for idx, i in enumerate(list(gene_lists.columns)):
    x = np.array(results_list[idx])[np.array(groups) == 'GTS']
    y = np.array(results_list[idx])[np.array(groups) == 'sport']
    t_test_gts_sport.append(stats.ttest_ind(x,y))
    x = np.array(results_list[idx])[np.array(groups) == 'GTS']
    y = np.array(results_list[idx])[np.array(groups) == '1kg']
    t_test_gts_1kg.append(stats.ttest_ind(x,y))
    x = np.array(results_list[idx])[np.array(groups) == 'sport']
    y = np.array(results_list[idx])[np.array(groups) == '1kg']
    t_test_sport_1kg.append(stats.ttest_ind(x,y))
    gts_mean.append(np.mean((np.array(results_list[idx])[np.array(groups) == 'GTS'])))
    gts_std.append(np.std(np.array(results_list[idx])[np.array(groups) == 'GTS']))
    kg_mean.append(np.mean((np.array(results_list[idx])[np.array(groups) == '1kg'])))
    kg_std.append(np.std(np.array(results_list[idx])[np.array(groups) == '1kg']))
    sport_mean.append(np.mean((np.array(results_list[idx])[np.array(groups) == 'sport'])))
    sport_std.append(np.std(np.array(results_list[idx])[np.array(groups) == 'sport']))

In [32]:
p_gts_sport = []

for t in t_test_gts_sport:
    p_gts_sport.append(t[1])
    
p_gts_kg = []

for t in t_test_gts_1kg:
    p_gts_kg.append(t[1])
    
p_kg_sport = []

for t in t_test_sport_1kg:
    p_kg_sport.append(t[1])

In [33]:
list_results = pd.DataFrame([list(gene_lists.columns), p_gts_sport, p_gts_kg, p_kg_sport, 
                             gts_mean, gts_std, kg_mean, kg_std, sport_mean, sport_std])

In [36]:
list_results

TypeError: 'Ttest_indResult' object is not callable

In [37]:
list_results.to_csv('list_results.csv')

In [15]:
results = per_gene.filter_rows(per_gene.gaba)

gaba_genes = results.within_gene(collect)

NameError: name 'collect' is not defined

In [ ]:
print(gaba_genes)

In [ ]:
len(gaba-genes)

In [ ]:
for g in gaba_genes:
    result = gaba_genes.filter_rows(gaba_genes.within_gene == g)
    print(gaba_genes.count())
    print('the gene is:'+g)
    r = result['burden_cadd_phred'].collect()
    print('burden score type - cadd phred results \n \t GTS vs sporstmen:')
    x = np.array(r)[np.array(groups) == 'GTS']
    y = np.array(r)[np.array(groups) == 'sport']
    print(stats.ttest_ind(x,y))
    print('\t GTS vs 1kg:')
    x = np.array(r)[np.array(groups) == 'GTS']
    y = np.array(r)[np.array(groups) == '1kg']
    print(stats.ttest_ind(x,y))
    print('\t sportsmen vs 1kg:')
    x = np.array(r)[np.array(groups) == 'sport']
    y = np.array(r)[np.array(groups) == '1kg']
    print(stats.ttest_ind(x,y))
    print('\t stats - mean & sd:')
    print('\t \t GTS - mean '+str(np.mean((np.array(r)[np.array(groups) == 'GTS'])))+' sd: '+str(np.std(np.array(e[idx][1])[np.array(groups) == 'GTS'])))
    print('\t \t sport - mean '+str(np.mean((np.array(r)[np.array(groups) == 'sport'])))+' sd: '+str(np.std(np.array(e[idx][1])[np.array(groups) == 'sport'])))
    print('\t \t 1kg - mean '+str(np.mean((np.array(r)[np.array(groups) == '1kg'])))+' sd: '+str(np.std(np.array(e[idx][1])[np.array(groups) == '1kg'])))
    print('\n')
    